In [ ]:
import tkinter as tk
import random
import math
import time
import threading


def get_heart_points(count, screen_width, screen_height, scale=25):
    points = []
    # 调整角度采样，让底部点分布更稀疏
    for i in range(count):
        # 原始均匀采样会导致底部点密集，改用非线性映射调整密度
        # t从0到2π，但底部区域（π附近）采样间隔变大
        ratio = i / (count - 1)  # 0到1的比例
        if ratio < 0.5:
            # 左半部分（0到π）：加速采样，减少底部点
            t = math.pi * (2 * ratio) ** 1.2
        else:
            # 右半部分（π到2π）：减速采样，保持对称
            t = math.pi + math.pi * (2 * (ratio - 0.5)) ** 0.8

        # 原始心形公式
        x = 16 * math.pow(math.sin(t), 3)
        y = 13 * math.cos(t) - 5 * math.cos(2 * t) - 2 * math.cos(3 * t) - math.cos(4 * t)

        center_x = screen_width // 2
        center_y = screen_height // 2
        # 微调y轴偏移，整体上移一点，减少底部堆积
        point_x = center_x + x * scale
        point_y = center_y - (y + 2) * scale  # y+1 轻微上移

        points.append((int(point_x), int(point_y)))
    return points


def show_warm_tip(pos, root, window_list):
    window = tk.Toplevel(root)
    window.update_idletasks()

    x, y = pos
    window_width = 200
    window_height = 40

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = max(0, min(x, screen_width - window_width))
    y = max(0, min(y, screen_height - window_height))

    window.title('温馨提示')
    window.geometry(f"{window_width}x{window_height}+{x}+{y}")

    tips = [
        '多喝水哦~', '记得按时吃饭哦', '每天都要元气满满',
        '记得吃水果', '保持好心情', '好好爱自己', '我想你了',
        '梦想成真', '期待下一次见面', '天天向上','立冬快乐！',
        '顺顺利利', '天冷照顾好自己呀~', '愿所有烦恼都消失',
        '早点睡，别熬夜', '今天过得开心嘛', '天冷了，多穿衣服'
    ]
    tip = random.choice(tips)

    bg_colors = [
        'lightpink', 'skyblue', 'lightgreen', 'lavender',
        'lightyellow', 'plum', 'coral', 'bisque', 'aquamarine'
    ]
    bg = random.choice(bg_colors)

    label = tk.Label(
        window,
        text=tip,
        bg=bg,
        font=('微软雅黑', 14),
        width=20,
        height=2
    )
    label.pack()
    label.update_idletasks()

    window.attributes('-topmost', True)
    window_list.append(window)
    return window


def spread_windows_instantly(window_list, screen_width, screen_height):
    time.sleep(1)
    window_width = 200
    window_height = 40
    for window in window_list:
        target_x = random.randint(0, screen_width - window_width)
        target_y = random.randint(0, screen_height - window_height)
        window.geometry(f"{window_width}x{window_height}+{target_x}+{target_y}")
        window.update()


if __name__ == "__main__":
    num_windows = 120  # 窗口总数保持不变
    root = tk.Tk()
    root.withdraw()

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    heart_points = get_heart_points(num_windows, screen_width, screen_height)
    window_list = []

    for i, pos in enumerate(heart_points):
        show_warm_tip(pos, root, window_list)
        time.sleep(0.08)
        if i % 5 == 0:
            root.update()

    spread_thread = threading.Thread(
        target=spread_windows_instantly,
        args=(window_list, screen_width, screen_height),
        daemon=True
    )
    spread_thread.start()

    root.mainloop()